In [ ]:
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

from enum import Enum
from copy import copy
from scipy.stats import norm
import pandas as pd
import numpy as np

import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))
from _utils.core_functions import *



In [ ]:
def calculate_target_risk_std(adjusted_close: pd.Series, current_price: pd.Series) -> float:
    """
    Calculates annualized volatility (standard deviation) of daily percentage returns
    over the most recent 30 business days.

    Uses current_price-relative returns based on adjusted_close and reference current_price series.
    """
    daily_returns = adjusted_close.diff() / current_price.shift(1)
    std = daily_returns.tail(30).std()
    return std * np.sqrt(BUSINESS_DAYS_IN_YEAR)


def calculate_variable_target_risk_std(
    adjusted_close: pd.Series,
    current_price: pd.Series,
    use_perc_returns: bool = True,
    annualise_stdev: bool = True,  
            
) -> pd.Series:
    """
    Computes a dynamic (variable) annualized volatility estimate using an
    exponentially weighted moving standard deviation of daily returns.

    Parameters
    ----------
    adjusted_close : pd.Series
        Adjusted close current_prices.
    current_price : pd.Series
        Reference current_price series.
    use_perc_returns : bool, default=True
        If True, computes daily percentage returns. If False, uses absolute differences.
    annualise_stdev : bool, default=True
        If True, scales standard deviation to annualized volatility using √252.

    Returns
    -------
    pd.Series
        Weighted annualized volatility estimate combining short-term (EWM)
        and long-term (10-year) volatility with weights 0.7 and 0.3 respectively.
    """
    if use_perc_returns:
        daily_returns = adjusted_close.diff() / current_price.shift(1)
    else:
        daily_returns = adjusted_close.diff()

    # Exponentially weighted daily standard deviation
    exp_daily_std = daily_returns.ewm(span=32).std()

    if annualise_stdev:
        annual_factor = np.sqrt(BUSINESS_DAYS_IN_YEAR)
    else:
        annual_factor = 1  # keep daily volatility

    annualized_std = exp_daily_std * annual_factor

    # 10-year ou max data rolling volatility mean
    std_long_term = annualized_std.rolling(BUSINESS_DAYS_IN_YEAR * 10, min_periods=1).mean()

    # Weighted combination (30% long-term, 70% recent)
    weighted_std = 0.3 * std_long_term + 0.7 * annualized_std

    return weighted_std


In [3]:
class standardDeviation(pd.Series):
    def __init__(
        self,
        adjusted_close: pd.Series,
        current_price: pd.Series,
        use_perc_returns: bool = True,
        annualise_stdev: bool = True,
    ):

        stdev = calculate_variable_target_risk_std(
            adjusted_close=adjusted_close,
            current_price=current_price,
            annualise_stdev=annualise_stdev,
            use_perc_returns=use_perc_returns,
        )
        super().__init__(stdev)

        self._use_perc_returns = use_perc_returns
        self._annualised = annualise_stdev
        self._current_price = current_price

    def daily_risk_price_terms(self):
        stdev = copy(self)
        if self.annualised:
            stdev = stdev / (BUSINESS_DAYS_IN_YEAR ** 0.5)

        if self.use_perc_returns:
            stdev = stdev * self.current_price

        return stdev

    def annual_risk_price_terms(self):
        stdev = copy(self)
        if not self.annualised:
            # daily
            stdev = stdev * (BUSINESS_DAYS_IN_YEAR ** 0.5)

        if self.use_perc_returns:
            stdev = stdev * self.current_price

        return stdev

    @property
    def annualised(self) -> bool:
        return self._annualised

    @property
    def use_perc_returns(self) -> bool:
        return self._use_perc_returns

    @property
    def current_price(self) -> pd.Series:
        return self._current_price



In [ ]:
def calculate_turnover(position, average_position):
    '''
    Return the Turnover defined as the number of times we turnover our average position
       
    '''
    daily_trades = position.diff()
    as_proportion_of_average = daily_trades.abs() / average_position.shift(1)
    average_daily = as_proportion_of_average.mean()
    annualised_turnover = average_daily * BUSINESS_DAYS_IN_YEAR

    return annualised_turnover

In [5]:
def calculate_position_series_given_variable_risk(
    capital: float,
    risk_target_tau: float,
    fx: pd.Series,
    multiplier: float,
    instrument_risk: standardDeviation,
) -> pd.Series:

    # N = (Capital × τ) ÷ (Multiplier × Price × FX × σ %)
    ## resolves to N = (Capital × τ) ÷ (Multiplier × FX × daily stdev price terms × 16)
    ## for simplicity we use the daily risk in price terms, even if we calculated annualised % returns
    daily_risk_price_terms = instrument_risk.daily_risk_price_terms()
    
    weigth_position_vol = capital * risk_target_tau / (multiplier * fx * daily_risk_price_terms * (BUSINESS_DAYS_IN_YEAR ** 0.5))

    return weigth_position_vol

In [6]:
DATA_DIR = '../_databases'
df_win = pd.read_excel((DATA_DIR + '/WINFUT.xlsx'),decimal=',',index_col='date')
df_win.sort_index(inplace=True)

df_wdo = pd.read_excel((DATA_DIR + '/WDOFUT.xlsx'),decimal=',',index_col='date')
df_wdo.sort_index(inplace=True)

In [7]:
adjusted_price = df_win['adjusted_close']
current_price = df_win['close']
multiplier = 0.2
risk_target_tau = 0.2
fx_series = pd.Series(1, index=df_win.index)  ## FX rate

capital = 100000


instrument_risk = standardDeviation(
    adjusted_close=adjusted_price,
    current_price=current_price,
    use_perc_returns=True,
    annualise_stdev=True,
)


In [8]:
position_contracts_held = calculate_position_series_given_variable_risk(
    capital=capital,
    fx=fx_series,
    instrument_risk=instrument_risk,
    risk_target_tau=risk_target_tau,
    multiplier=multiplier,
)


In [ ]:
perc_return = calculate_perc_returns(
       position_contracts_held=position_contracts_held,
       adjusted_price=adjusted_price,
       fx=fx_series,
       capital=capital,
       multiplier=multiplier,
   )

In [10]:
# --- Plot retorno acumulado
fig = go.Figure()

fig.add_trace(
    go.Scatter(
        x=perc_return.index,
        y=perc_return.cumsum()*100,
        name="WINFUT - Estratégia com ajuste de risco",
        line=dict(color="blue"),
    )
)

fig.update_layout(
    title="Estratégia com Ajuste de Risco - alvo std 10%- Retorno Acumulado",
    xaxis_title="Data",
    yaxis_title="Retorno Acumulado (%)",
    template="plotly_white",
    showlegend=True,
    legend=dict(
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1),
    height=600,
    width=1000,
)

fig.show()

# --- Exibir estatísticas
print("Estatísticas de Desempenho - WIN")
print("Estatísticas na Frequência Diária")
print(calculate_stats(perc_return))
print("================================")
print("Estatísticas na Frequência Mensal")
print(calculate_stats(perc_return, freq=MONTH))
print("================================")
print("Capital Mínimo Necessário para 100 Contratos de WIN")
print(
    calculate_minimum_capital(
        multiplier=multiplier,
        target_risk=risk_target_tau,
        fx=1,
        ann_volatility=instrument_risk[-1],
        price=current_price.iloc[-1],
        n_contracts=100,
    )
)

print("Turnover WIN")
print(
        calculate_turnover(
            position_contracts_held, average_position=position_contracts_held
        )
    )

Estatísticas de Desempenho - WIN
Estatísticas na Frequência Diária
{'ann_mean': 0.019758389372973906, 'ann_std': 0.17525942736035363, 'sharpe': 0.11273795464564867, 'skew': -0.23354615659763941, 'avg_drawdown': 0.26554609851464284, 'max_drawdown': 0.7431347183688913, 'quant_lower': 1.2793692962526697, 'quant_upper': 1.2267277773398113}
Estatísticas na Frequência Mensal
{'ann_mean': 0.4071352672016453, 'ann_std': 0.6878491346017331, 'sharpe': 0.5918961683907299, 'skew': -0.18397358021474927, 'avg_drawdown': 0.2540732798336465, 'max_drawdown': 0.6799022738324277, 'quant_lower': 0.9379973100368431, 'quant_upper': 0.809406236186574}
Capital Mínimo Necessário para 100 Contratos de WIN
2787565.7695328426
Turnover WIN
5.32995815808431
